# Initialisation

## Jeu de données

Nous allons vous présenter l'utilisation des algorithmes backward, forward et viterbi à l'aide d'un jeu de données que nous avons programmée en suivant le schéma ci-dessous.

<img src="Donnees.png"/>

D'abord, une importation de la librairie numpy est nécessaires

In [1]:
import numpy as np

Nous allons générer 5 états que nous allons utiliser et analyser tout le long de nos programmes.  

In [2]:
N = 5 #nombre d'états à generer

Ensuite, on prend des probabilités aléatoires pour construire la matrice des états et des observations en suivant les probabilités indiquées dans le schéma au dessus. 

## Etat 0

Suivant les probabilités connues, nous avons généré un premier état : 

In [3]:
etats = []

if  np.random.rand() < 0.6:
    etats.append('Sain')
else:
    etats.append('Malade')

## Observation 0

Une nouvelle fois, nous allons générer un nombre entre 0 et 1 de manière aléatoire afin d'avoir la probabilité d'une première observation. 

In [4]:
obs = []

tmp = np.random.rand()

En suivant le schéma nous avons : 

    Si sain : 
        70% normal
        20% rhume 
        10% fievre

    Si malade : 
        10% normal
        40% rhume
        50% fievre

Ainsi, nous avons construit un programme permettant d'obtenir l'observation à l'état 0 : 

In [5]:
if etats[0] == 'Sain':
    if tmp < 0.7:
        obs.append('Normal')
    elif tmp < 0.9:
        obs.append('Rhume')
    else:
        obs.append('Fièvre')
else:
    if tmp < 0.1:
        obs.append('Normal')
    elif tmp < 0.5:
        obs.append('Rhume')
    else:
        obs.append('Fièvre')

## Liste des états et des observations

Grâce aux programmes fait au préalable nous avons l'état et l'observation à l'instant 0. De ce fait, nous pouvons générer les observations et les états des instants suivant. 

Génération des états : 

    Si état k est sain : 
        70% que l'état k+1 soit sain 
        30% que l'état k+1 soit malade 
    Sinon :
        40% que l'état k+1 soit sain 
        60% que l'état k+1 soit malade 
        
Génération des observations : 
        
    Si l'état k+1 est sain : 
        70% normal 
        20% rhume 
        10% fièvre 
        
    Sinon : 
        10% normal 
        30% rhume 
        60% fièvre 

In [6]:
for k in range(N-1):
    if etats[k] == 'Sain':
        if np.random.rand() < 0.7:
            etats.append('Sain')
        else:
            etats.append('Malade')
    else:
        if np.random.rand() < 0.6:
            etats.append('Malade')
        else:
            etats.append('Sain')

    if etats[k+1] == 'Sain':
        if tmp < 0.7:
            obs.append('Normal')
        elif tmp < 0.9:
            obs.append('Rhume')
        else:
            obs.append('Fièvre')
            
    else:
        if tmp < 0.1:
            obs.append('Normal')
        elif tmp < 0.4:
            obs.append('Rhume')
        else:
            obs.append('Fièvre')

# FORWARD-BACKWARD

L'algorithme forward-backward permet de trouver les états les plus probables, pour un modèle de Markov caché, à n'importe quel instant. 

Nous créons une matrice de transition (2x2) qui nous permet d'avoir les probabilités des états entre eux (par exemple il y a 70% de chance que si un patient est sain à t il le restera à t+1) : 

In [7]:
Transmi = np.array([[0.7 , 0.3],[0.4 , 0.6]])
print(Transmi)

[[0.7 0.3]
 [0.4 0.6]]


## Forward

L'algorithme de forward permet d'avoir les premières observations dans une séquence donnée. C'est-à-dire on obtient la probabilité d'un état à un certain moment, compte tenu de l'historique des preuves. Il va de la premoère séquence à la dernière. 

Génération d'une matrice d'observation à l'état 0 : 

In [8]:
if obs[0] == 'Normal' : 
    OF = np.array([[0.7 , 0] , [0 , 0.1]])
elif obs[0] == 'Rhume' : 
    OF = np.array([[0.2 , 0] , [0 , 0.4]])
else : 
    OF = np.array([[0.1 , 0] , [0 , 0.5]])
    
print(OF)

[[0.7 0. ]
 [0.  0.1]]


Ici, en fonction de l'observation à l'état 0 nous obtenons une matrice contenant les probabilités pour l'observation sélectionné en fonction des 2 états existants (si l'observation est normal, nous prenons la probabilité liant l'état sain et normal ainsi que l'état malade et normal) dont nous avons besoin pour la suite. 

De ce fait, nous calculons la probabilité à l'instant 0 en faisant le produit scalaire entre les probabilités initiales (sain (60%) ou malade (40%)) et la matrice d'observation généré au-dessus : 

In [9]:
PinitF = [0.6 , 0.4]
PF = np.zeros([N,2])

PF[0,:] = np.dot(PinitF,OF) 
print(PF)

[[0.42 0.04]
 [0.   0.  ]
 [0.   0.  ]
 [0.   0.  ]
 [0.   0.  ]]


Ainsi nous pouvons créer l'algorithme de forward : 

Si l'observation à k+1 est normal : 
    la matrice d'observation prends les probabilités de transmission de normal vers sain et malade
Si l'observation à k+1 est rhume : 
    la matrice d'observation prends les probabilités de transmission de rhume vers sain et malade
Si l'observation à k+1 est malade : 
    la matrice d'observation prends les probabilités de transmission de malade vers sain et malade
    
De ce fait, pour calculer la probabilité à l'instant k+1 nous faisons un produit scalaire entre la probabilité à k, la matrice de transmission (entre les états) définit plus tôt et la matrice d'observation obtenu juste au dessus. Chaque probabilité calculée sera utilisé pour calculer la probabilité de l'observation suivante. 

In [10]:
for k in range(N-1):
    if obs[k+1] == 'Normal' : 
        OF = np.array([[0.7 , 0] , [0 , 0.1]])
    elif obs[k+1] == 'Rhume' : 
        OF = np.array([[0.2 , 0] , [0 , 0.4]])
    else : 
        OF = np.array([[0.1 , 0] , [0 , 0.5]])
    PF[k+1,:] = np.dot(PF[k,:],np.dot(Transmi,OF))
    
print(PF)

[[0.42       0.04      ]
 [0.217      0.015     ]
 [0.11053    0.00741   ]
 [0.0562345  0.0037605 ]
 [0.02860784 0.00191266]]


Ainsi nous avons les probabilités d'obtention des états. 

Par exemple, nous avons 6% de chance que l'état à l'instant 0 soit sain. 

## Backward

L'algorithme de backward est un calcul rétrogressif des probabilités. C'est-à-dire la probabilité d'obtenir les autres observations ultérieurs à un état initial. Nous repartons maintenant de la dernière observation vers la première. Une nouvelle fois, chaque probabilité calculée sera utilisé pour calculer la probabilité de l'observation précédante. 

Génération d'une matrice d'observation à l'état 0 : 

In [11]:
if obs[N-1] == 'Normal' : 
    OB = np.array([[0.7 , 0] , [0 , 0.1]])
elif obs[N-1] == 'Rhume' : 
    OB = np.array([[0.2 , 0] , [0 , 0.4]])
else : 
    OB = np.array([[0.1 , 0] , [0 , 0.5]])

Cette matrice suit le même principe que la matrice d'observation utilisé pour l'algorithme forward. 

Ainsi nous calculons la probabilité au dernier instant (k=4 pour python) en faisant le produit scalaire entre la matrice de transition, d'observation et les probabilités initiales (1 et 1 car il n'y a pas de probabilité initiale concernant la fin des générations des états) : 

In [12]:
PinitB = [1 , 1]
PB = np.zeros([N,2])
    
PB[N-1,:] = np.dot(Transmi, np.dot(OB,PinitB))
PB

array([[0.  , 0.  ],
       [0.  , 0.  ],
       [0.  , 0.  ],
       [0.  , 0.  ],
       [0.52, 0.34]])

De ce fait, nous pouvons calculer l'algorithme de backward. En prenant en compte l'observation k+1 nous obtenons la matrice d'observation en suivant toujours le même principe : si l'observation est rhume, nous prenons la probabilité liant l'état sain et rhume ainsi que l'état malade et rhume. 

Ainsi, nous calculons les probabilités pour les deux états à l'instant k en effectuant le produit scalaire entre la matrice de transmission, la matrice d'observation obtenue au dessus et les probabilités des états à k+1. 

In [13]:
for k in range(N-2,-1,-1):
    if obs[k+1] == 'Normal' : 
        OB = np.array([[0.7 , 0] , [0 , 0.1]])
    elif obs[k+1] == 'Rhume' : 
        OB = np.array([[0.2 , 0] , [0 , 0.4]])
    else : 
        OB = np.array([[0.1 , 0] , [0 , 0.5]])
    PB[k,:] = np.dot(Transmi, np.dot(OB,PB[k+1]))
    
print(PB)

[[0.03489389 0.02177374]
 [0.0685915  0.042802  ]
 [0.13483    0.08416   ]
 [0.265      0.166     ]
 [0.52       0.34      ]]


Ainsi nous avons les probabilités d'obtention des états. 

Par exemple, nous avons 0.2% de chance que l'état à l'instant 0 soit sain. 

### BACKWARD-FORWARD

Dorénavant, on peut obtenir la probabilité de l'état caché en prenant en compte la séquence totale des évènements observés. Nous obtenons un résultat plus précis grâce à toutes les observations effectuées. Ainsi, nous multiplions les probabilités que nous avons trouvé avec les deux algorithmes. 

In [14]:
PBF = PF * PB
print(PBF)   

[[0.01465544 0.00087095]
 [0.01488436 0.00064203]
 [0.01490276 0.00062363]
 [0.01490214 0.00062424]
 [0.01487608 0.00065031]]


Ainsi, nous pouvons voir quel sont les états les plus probables. Par exemple, à l'instant k, si la probabilité pour l'état sain est plus grande que pour l'état malade, il y a plus de chance qu'il sera sain à l'instant k.

In [15]:
etatfinal = list()

if PBF[0,0] > PBF[0,1] : 
    etatfinal.append('sain')
else :
    etatfinal.append('malade')

for k in range(N-1):
    if PBF[k,0] > PBF[k,1] : 
        etatfinal.append('sain')
    else :
        etatfinal.append('malade')

print(etatfinal)

['sain', 'sain', 'sain', 'sain', 'sain']


Ainsi, on peut voir quel état est le plus probable pour les 5 instants. Ici, il y a que l'état sain qui ressort. 

# VITERBI

L'algortihme de viterbi permet de trouver la séquence d'états la plus probable en donnant l'estimation par maximum a posteriori. En effet, elle récupère la séquence d'états pour laquelle la probabilité a posteriori est maximale. 

Pour commencer, nous générons une matrice d'observation à l'état 0  en éxécutant un code suivant le même principe suivi pour les deux autres algorithmes. 

In [16]:
if obs[0] == 'Normal' : 
    OV = np.array([[0.7 , 0] , [0 , 0.1]])
elif obs[0] == 'Rhume' : 
    OV = np.array([[0.2 , 0] , [0 , 0.4]])
else : 
    OV = np.array([[0.1 , 0] , [0 , 0.5]])

## Matrice des probabilités maximales et des états précédants : 

Dorénavant, nous calculons la probabilité à l'état 0 en faisant le produit scalaire entre la matrice de transition et les probabilités initiales (probabilité d'être sain ou malade). Enfin, nous pouvons trouver quelle est la probabilité maximum entre celle obtenu pour que l'état soit sain et pour que l'état soit malade afin de récupérer la plus forte c'est-à-dire celle qui a le plus de chance de s'éxécuter. 

In [17]:
PinitV = [0.6 , 0.4]
PV = np.zeros([N,2])
ProbaMax = np.zeros([N,1])

PV[0,:] = np.dot(PinitV,Transmi)
ProbaMax[0] = max(PV[0,0],PV[0,1])

Ainsi, nous pouvons calculer la probabilité d'un état en récupérant le maximum entre la probabilité qu'il soit sain et qu'il soit malade. Nous effectuons ce calcul en deux parties : d'abord pour la séquence d'état sain puis pour celle de l'état malade. Grâce à celà et à un maximum effectué entre ces deux probabilités calculées nous pouvons récupérer l'état le plus probable (dans la deuxième partie). 

D'abord, nous calculons à nouveau la matrice d'observation à chaque séquence.

Pour calculer la probabilité de l'état sain nous prenons le maximum entre : 
    - le  produit scalaire de la probabilité d'avoir l'état sain a posteriori, la matrice de transimission de sain à sain et la matrice d'observation pour l'état sain. 
    - le  produit scalaire de la probabilité d'avoir l'état malade a posteriori, la matrice de transimission de sain à malade et la matrice d'observation pour l'état sain. 

Pour calculer la probabilité de l'état malade nous prenons le maximum entre : 
    - le  produit scalaire de la probabilité d'avoir l'état sain a posteriori, la matrice de transimission de malade à sain et la matrice d'observation pour l'état malade. 
    - le  produit scalaire de la probabilité d'avoir l'état malade a posteriori, la matrice de transimission de malade à malade et la matrice d'observation pour l'état malade. 
    
Nous calculons la probabilité maximale entre ces deux probabilités calculées. 

Pour finir, après avoir créé la matrice de stockage, dedans, on génère par des 0 et 1 le maximum calculé au dessus (celui pour l'état sain et pour l'état malade : pas ProbaMax). Par exemple, si à la première ligne, première colonne il affiche 1 cela veut dire que même si il est sain, il était malade à l'état précédant. 

In [18]:
Stockage = np.zeros([N,2])

for k in range(N-1):
    if obs[k+1] == 'Normal' : 
        OV = np.array([[0.7 , 0] , [0 , 0.1]])
    elif obs[k+1] == 'Rhume' : 
        OV = np.array([[0.2 , 0] , [0 , 0.4]])
    else : 
        OV = np.array([[0.1 , 0] , [0 , 0.5]])
    PV[k+1,0] = max(np.dot(PV[k,0],(np.dot(Transmi[0,0], OV[0,0]))),np.dot(PV[k,1],(np.dot(Transmi[0,1], OV[0,0]))))
    PV[k+1,1] = max(np.dot(PV[k,0],(np.dot(Transmi[1,0], OV[1,1]))),np.dot(PV[k,1],(np.dot(Transmi[1,1], OV[1,1]))))
    ProbaMax[k+1] = max(PV[k+1,0],PV[k+1,1])
    
    Stockage[k,0] = np.argmax([np.dot(PV[k-1,0],(np.dot(Transmi[0,0], OV[0,0]))),np.dot(PV[k-1,1],(np.dot(Transmi[0,1], OV[0,0])))])
    Stockage[k,1] = np.argmax([np.dot(PV[k-1,1],(np.dot(Transmi[1,0], OV[1,1]))),np.dot(PV[k-1,1],(np.dot(Transmi[1,1], OV[1,1])))])
            
print(PV)

print(ProbaMax)

[[0.58       0.42      ]
 [0.2842     0.0252    ]
 [0.139258   0.011368  ]
 [0.06823642 0.00557032]
 [0.03343585 0.00272946]]
[[0.58      ]
 [0.2842    ]
 [0.139258  ]
 [0.06823642]
 [0.03343585]]


Avant de voir la matrice des états les plus probables, nous pouvons rapidement voir quel est l'état antérieur de chaque état. 

Pour cela, nous prenons notre matrice de stockage et lorsqu'elle affichait 0 nous avons remplacé cela par sain et si elle affichait 1 nous l'avons remplacé par malade. Pour cela, nous avons créé deux listes, une pour l'état sain et l'autre pour l'état malade. 

In [19]:
print(Stockage)

etatsanterieur = list() 
etatsanterieurM = list() 

#sain    
for k in range(N):
    if Stockage[k,0] == 1 : 
        etatsanterieur.append('Malade')
    else : 
        etatsanterieur.append('Sain')

#malade
for k in range(N):
    if Stockage[k,1] == 1 : 
        etatsanterieurM.append('Malade')
    else : 
        etatsanterieurM.append('Sain')

[[0. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 0.]]


Les états précédants les plus probables quand on est sain sont : 

In [20]:
print(etatsanterieur)

['Sain', 'Sain', 'Sain', 'Sain', 'Sain']


Les états précédants les plus probables quand on est malade sont : 

In [21]:
print(etatsanterieurM)

['Sain', 'Malade', 'Malade', 'Malade', 'Sain']


## Matrice des états les plus probables : 

Comme dit précédemment, nous souhaitons dorénavant récupérer les états les plus probables.

Tout d'abord, nous générons l'état le plus probable pour le dernier état récupéré avant de les trouver grâce à une boucle.

Comme pour calculer la probabilité maximale, nous prenons la maximum entre la probabilité d'être sain (ligne 0) et celle d'être malade (ligne 1), en utilisant les probabilités calculées juste au dessus (PV), et générons ce résultat sous force de 0 (sain) ou 1 (malade). Ainsi, nous pouvons récupérer dans la matrice de stockage quel est l'état le plus probable pour l'instant k en récupérant sa donnée placé pour la séquence k+1.  

De plus, nous créons une liste afin de visualier ce que nous venons de faire et qui part du dernier état le plus probable au premier état le plus probable. Celle-ci traduit les 0 et les 1 en malade et sain. Elle créé la chaîne des états. 

In [22]:
etats_probables = np.zeros([N,1])
etatspp = list()

#son état à la fin : 
if PV[N-1,0] > PV[N-1,1] : 
    etats_probables[N-1] = 0
else :
    etats_probables[N-1] = 1

for k in range(N-2,-1,-1) :
    if etats_probables[k+1] == 1 : 
        etats_probables[k] = Stockage[k+1,1]
        if etats_probables[k] == 0 : 
            etatspp.append('Sain')
        else : 
            etatspp.append('Malade')
    else : 
        etats_probables[k] = Stockage[k+1,0]
        if etats_probables[k] == 0:
            etatspp.append('Sain')
        else : 
            etatspp.append('Malade')
            
if etats_probables[0] == 0:
    etatspp.append('Sain')
else : 
    etatspp.append('Malade')
        
print(etats_probables)
print(etatspp)

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
['Sain', 'Sain', 'Sain', 'Sain', 'Sain']
